# Семинар 14

# Метод Ньютона: дорого и быстро

## На прошлом семинаре...

1. Методы спуска
2. Направление убывания
3. Градиентный метод
4. Правила выбора шага
5. Теоремы сходимости
6. Эксперименты

## Недостатки градиентного спуска

- Линейная скорость сходимости
- Зависимость от числа обусловнности гессиана

Можно ли их одновременно преодолеть?

## Мотивация метода Ньютона
Рассмотрим задачу
$$
\min\limits_{x\ \in \mathbb{R}^n} f(x).
$$

- Градиентный спуск $\equiv$ **линейная** аппроксимация $f$
- Метод Ньютона $\equiv$ **квадратичная** аппроксимация $f$:
$$
f(x + h) \approx f(x) + \langle f'(x), h \rangle + \frac{1}{2}h^{\top}f''(x)h \to \min_{h}
$$
Из необходимого условия минимума:
$$
f'(x) + f''(x) h = 0, \qquad h^* = -(f''(x))^{-1} f'(x)
$$


## Метод Ньютона

```python
def NewtonMethod(f, x0, epsilon, **kwargs):
    x = x0
    while True:
        h = ComputeNewtonStep(x, f, **kwargs)
        if StopCriterion(x, f, h, **kwargs) < epsilon:
            break
        alpha = SelectStepSize(x, h, f, **kwargs)
        x = x + alpha * h
    return x

```

## Теорема сходимости

**Теорема.** Пусть функция $f(x)$ такая что
- дважды дифференцируема и гессиан удовлетворяет условию Липшица с константой $M$
- существует точка локального минимума с положительно определённым гессианом
$$
f''(x^*) \succeq lI, \; l > 0
$$
- начальная точка $x_0$ достаточно близка к точки минимума, в частности
$$
\|x_0 - x^*\|_2 \leq \frac{2l}{3M}
$$

Тогда метод Ньютона сходится **квадратично**:
$$
\|x_{k+1} - x^* \|_2 \leq \dfrac{M\|x_k - x^*\|^2_2}{2 (l - M\|x_k - x^*\|_2)}
$$

## Афинная инвариантность

Рассмотрим функцию $g(y) = f(Ax)$, где $A$ - матрица невырожденного преобразования. Тогда
$$
g(y + u) \approx g(y) + \langle g'(y), u \rangle + \frac{1}{2} u^{\top} g''(y) u 
$$
и
$$
u^* = -(g''(y))^{-1} g'(y)
$$
или
$$
u^* = -(A^{\top}f''(y) A)^{-1} A^{\top}f'(y) = -A^{-1} (f''(y))^{-1}f'(y) = -A^{-1}h^*
$$

Таким образом, направление по методу Ньютона преобразуется при линейном преобразовани так же, как и координаты!

## Эксперименты

## Pro & Contra

Pro

- квадратичная сходимость вблизи решения
- афинная инвариантность
- параметры слабо влияют на скорость сходимости

Contra

- необходимо хранить гессиан на каждой итерации: $O(n^2)$ памяти
- необходимо решать линейные системы: $O(n^3)$ операций

## Резюме

1. Метод Ньютона
2. Теоремы сходимости
4. Сравнение с градиентным спуском
5. Эксперименты